### Figure 1: Predicted images

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import metpy.plots.ctables
from utils.data import load_tc_data


sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))
sns.set_palette(sns.color_palette("Set2"))

In [2]:
# precip_clevs = [0, 1, 2.5, 5, 7.5, 10, 15, 20, 30, 40,
#      50, 70, 100, 150, 200, 250, 300, 400, 500, 600, 750, 1000]
# precip_norm, precip_cmap = metpy.plots.ctables.registry.get_with_boundaries('precipitation', precip_clevs)
precip_clevs = [0, 0.1, 1, 2.5, 5, 7.5, 10, 15, 20, 30, 40, 50, 70, 100, 150, 200]
precip_clevs = [0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 70, 100, 150]
precip_cmap = matplotlib.colors.ListedColormap(metpy.plots.ctables.colortables["precipitation"][:len(precip_clevs)-1], 'precipitation')
precip_norm = matplotlib.colors.BoundaryNorm(precip_clevs, precip_cmap.N)

tc_colours = [(255/255,255/255,255/255), # no rain
			(169/255, 209/255, 222/255), # drizzle 0-1
			(137/255, 190/255, 214/255), # drizzle 1-2
			(105/255, 160/255, 194/255), # drizzle 2-3
			# (106/255, 173/255, 146/255), # drizzle 3-5
			(93/255, 168/255, 98/255), # drizzle 3-5
			# (93/255, 168/255, 98/255), # very light rain 5-7
			(128/255, 189/255, 100/255), # very light rain 5-7
			# (128/255, 189/255, 100/255), # light rain 7-10
			(165/255, 196/255, 134/255), # light rain 7-10
			(233/255, 245/255, 105/255), # rain 10-15
			(245/255, 191/255, 105/255), # heavy rain 15-20
			(245/255, 112/255, 105/255), # heavier rain 20-25
			(245/255, 105/255, 149/255), # real heavy rain 25-30
			# (227/255, 113/255, 182/255), # intense rain 30-40
			(240/255, 93/255, 154/255), # intense rain 30-40
			# (154/255, 91/255, 235/255), # super intense rain 40-50
			(194/255, 89/255, 188/255), # super intense rain 40-50
			(66/255, 57/255, 230/255), # insane amount of rain 50-70
			(24/255, 17/255, 153/255), # you do not want to be caught in this rain 70-100
			(9/255, 5/255, 87/255), # I can't belive the scle goes up this high 100-150
]

precip_cmap = LinearSegmentedColormap.from_list('tc_colours',tc_colours,N=16) #increasing N makes it smoother
precip_norm = matplotlib.colors.BoundaryNorm(precip_clevs, precip_cmap.N)

In [3]:
def regrid(array):
        hr_array = np.zeros((100,100))
        for i in range(10):
                for j in range(10):
                        i1 = i*10
                        i2 = (i+1)*10
                        j1 = j*10
                        j2 = (j+1)*10
                        hr_array[i1:i2,j1:j2] = array[i,j]
        return hr_array

In [4]:
def plot_predictions(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_,plot='save'):
	if plot == 'save':
			fig, axes = plt.subplots(n, m, figsize=(5*m, 5*n), sharey=True)
	else:
			print('show')
			fig, axes = plt.subplots(n, m, figsize=(2*m, 2*n), sharey=True)

	pal = sns.cubehelix_palette(10, rot=-.25, light=.7,as_cmap=True)

	for i in range(m):
		j = 0
		
		storm = storms[i]
		
		print('storm max: ',real[storm].max())
		print('cnn max: ',pred_cnn[storm].max())
		print('vaegan max: ',pred_vaegan[storm].max())
		print('gan max: ',pred_gan[storm].max())
		print('inputs max: ', np.nanmax(inputs[storm]))
		# norm = colors.Normalize(*range_)
		norm = precip_norm
		if i in [0,1,2]:
			axes[j,i].imshow(regrid(inputs[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j+1,i].imshow(pred_cnn[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].imshow(pred_vaegan[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].imshow(pred_gan[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			im = axes[j+4,i].imshow(real[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
		else:
			axes[j,i].imshow(regrid(inputs_x[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j+1,i].imshow(pred_cnn_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].imshow(pred_vaegan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].imshow(pred_gan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			im = axes[j+4,i].imshow(real_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)

		axes[j,i].set(xticklabels=[])
		axes[j,i].set(yticklabels=[])
		axes[j+1,i].set(xticklabels=[])
		axes[j+1,i].set(yticklabels=[])
		axes[j+2,i].set(xticklabels=[])
		axes[j+2,i].set(yticklabels=[])
		axes[j+3,i].set(xticklabels=[])
		axes[j+3,i].set(yticklabels=[])
		axes[j+4,i].set(xticklabels=[])
		axes[j+4,i].set(yticklabels=[])


		axes[j,0].set_ylabel('LR input',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+1,0].set_ylabel('U-NET',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+2,0].set_ylabel('VAEGAN',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+3,0].set_ylabel('GAN',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+4,0].set_ylabel('HR obs',horizontalalignment='left',rotation=0,fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')

		axes[4,j].set_xlabel('Maria (2017; NA)',fontsize=24,labelpad=5)
		axes[4,j+1].set_xlabel('Haiyan (2013; WP)',fontsize=24,labelpad=5)
		axes[4,j+2].set_xlabel('Barbara (1995; EP)',fontsize=24,labelpad=5)
		axes[4,j+3].set_xlabel('Georges (1998; NA)',fontsize=24,labelpad=5)
		axes[4,j+4].set_xlabel('Daman (1992; SP)',fontsize=24,labelpad=5)
		axes[4,j+5].set_xlabel('Haleh (2019; SI)',fontsize=24,labelpad=5)

	fig.subplots_adjust(right=0.8)
	# cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
	cbar_ax = fig.add_axes([1, 0.031, 0.03, 0.936])
	cb = fig.colorbar(im, cax=cbar_ax,shrink=0.01)
	cb.ax.tick_params(labelsize=36)
	fig.tight_layout()

	if plot == 'save':
			plt.savefig('figure_1.png',bbox_inches='tight')
			plt.clf()
	else:
			plt.show()
	

In [5]:
# load data
real,inputs,pred_cnn,pred_vaegan,pred_gan,pred_vaegan_ensemble,pred_gan_ensemble,meta = load_tc_data(set='validation',results='test')
real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_valid',results='final')

meta = pd.read_csv('/user/work/al18709/tc_data_mswep/valid_meta.csv')
meta_extreme = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_valid_meta.csv')
meta_extreme_test = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_test_meta.csv')
meta_test = pd.read_csv('/user/work/al18709/tc_data_mswep/test_meta.csv')
meta_train = pd.read_csv('/user/work/al18709/tc_data_mswep/train_meta.csv')

In [6]:

plot = 'save'
n = 5
m = 6
# make colours
range_ = (0, 20)
# storms = [16249,15872,3,4,5,6,7,8]
storms = [4077, 4078, 4079, 4080, 4081, 4082]

# maria
# 8453 (8452 for unet)
# 8463

# haiyan
# 9809
# 9815, 9816, 9822
# 9820, 9828,  9845, 9840

# barb 1347 1375 1399

# georges 4082
storms = [8453,9845,1375,4128,5358,747]
# storms_extremes = [4082,5349,763]

plot_predictions(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_)


storm max:  45.25
cnn max:  29.72098
vaegan max:  43.15578079223633
gan max:  51.5527229309082
inputs max:  20.504125595092773
storm max:  58.875
cnn max:  15.141447
vaegan max:  57.22317123413086
gan max:  61.957401275634766
inputs max:  31.177553176879883
storm max:  48.3125
cnn max:  37.28275
vaegan max:  60.50486373901367
gan max:  60.7703971862793
inputs max:  29.4201602935791
storm max:  44.375
cnn max:  31.241987
vaegan max:  48.12854766845703
gan max:  42.21123123168945
inputs max:  25.278505325317383
storm max:  59.9375
cnn max:  41.355465
vaegan max:  49.47930908203125
gan max:  55.63440704345703
inputs max:  31.395225524902344
storm max:  44.8125
cnn max:  38.602913
vaegan max:  37.492549896240234
gan max:  47.49197769165039
inputs max:  20.156835556030273


/user/work/al18709/.conda/envs/aquatic/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


<Figure size 2160x1800 with 0 Axes>

In [7]:
pred_vaegan = np.load('/user/home/al18709/work/vaegan_predictions_20/validation_pred-opt_3_better-noise.npy')
pred_vaegan_x = np.load('/user/home/al18709/work/vaegan_predictions_20/extreme_valid_pred-opt_3_better-noise.npy')
# pred_gan_x = np.load('/user/home/al18709/work/gan_predictions_20/extreme_valid_pred-3_better-noise.npy')
# pred_gan = np.load('/user/home/al18709/work/gan_predictions_20/validation_pred-opt_3_better-noise.npy')
range_ = (0, 20)
storms = [8453,9845,1375,4128,5358,747]


plot_predictions(pred_vaegan[:,:,:,0],pred_vaegan_x[:,:,:,0],pred_vaegan[:,:,:,1],pred_vaegan_x[:,:,:,2],pred_vaegan[:,:,:,2],pred_vaegan_x[:,:,:,2],pred_vaegan[:,:,:,3],pred_vaegan_x[:,:,:,3],pred_vaegan[:,:,:,4],pred_vaegan_x[:,:,:,4],storms,range_)
# plot_predictions(pred_gan[:,:,:,0],pred_gan_x[:,:,:,0],pred_gan[:,:,:,1],pred_gan_x[:,:,:,2],pred_gan[:,:,:,2],pred_gan_x[:,:,:,2],pred_gan[:,:,:,3],pred_gan_x[:,:,:,3],pred_gan[:,:,:,4],pred_gan_x[:,:,:,4],storms,range_)

MemoryError: Unable to allocate 24.2 GiB for an array with shape (3250600000,) and data type float64

In [6]:
pred_gan = np.load('/user/home/al18709/work/gan_predictions_20/validation_pred-opt_5_normal_problem.npy')[:,:,:,0]

In [ ]:
# print(meta.shape)
# print(real.shape) 

# print(np.sum(meta['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme['sid'] == '1992044S09181'))
# print(np.sum(meta_test['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme_test['sid'] == '1992044S09181'))
# print(np.sum(meta_train['sid'] == '1992044S09181'))


# print(meta_extreme['sid'].values)
# for i,storm in enumerate(meta_extreme['sid'].values):
# 	pre_storm = meta_extreme['sid'].values[i-1]
# 	if storm != pre_storm:
# 		print(storm)

# Dorian in the test set
# Isaac (2012) '2012234N16315' training set NA
# Amphan 2020136N10088 ****** lost NI
# Harvey 2017228N14314 train NA
# Irma 2017242N16333 train NA
# Idai 2019063S18038 train SIO 
# Winston 2016041S14170 test SP
# Matthew 2016273N13300 extreme_test NA
# Jalawat 2012263N15141 extreme_test WP


# EP NA WP
# NIO SIO SP

# Maria 2017260N12310 validation NA
# Haiyan 2013306N07162 validation WP
# Barbara 1995189N11257 validation EP
print(np.where(meta_extreme['sid'] == '2019060S10078'))

# Georges 1998259N10335 extreme valid NA
# Daman 1992044S09181 extreme valid SP
# Haleh 2019060S10078 extreme valid SIO